In [1]:
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup

In [3]:
url=input("Enter the Product URL: ")

In [4]:
#r=requests.get('https://www.yelp.com/biz/tesla-san-francisco?osq=Tesla+Dealership')
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
r=requests.get(url,headers=headers)


In [5]:
r.status_code

200

In [6]:
soup = BeautifulSoup(r.text,'html.parser')


In [7]:
# divs = soup.findAll(class_="css-1qn0b6x")
review_content = soup.findAll("div",{'data-hook':'review'})
review_content

[<div class="a-section review aok-relative" data-hook="review" id="R2NMSFGTG9O7O8"><div class="a-row a-spacing-none" id="R2NMSFGTG9O7O8-review-card"><div class="a-section celwidget" id="customer_review-R2NMSFGTG9O7O8"><div class="a-row a-spacing-mini" data-hook="genome-widget"><a class="a-profile" data-a-size="small" href="/gp/profile/amzn1.account.AF324DE6FRMO6D5LTOCIPUG4SDGA/ref=cm_cr_dp_d_gw_tr?ie=UTF8"><div aria-hidden="true" class="a-profile-avatar-wrapper"><div class="a-profile-avatar"><img class="a-lazy-loaded" data-src="https://images-eu.ssl-images-amazon.com/images/S/amazon-avatars-global/default._CR0,0,1024,1024_SX48_.png" src="https://images-na.ssl-images-amazon.com/images/G/01/x-locale/common/grey-pixel.gif"/><noscript><img src="https://images-eu.ssl-images-amazon.com/images/S/amazon-avatars-global/default._CR0,0,1024,1024_SX48_.png"/></noscript></div></div><div class="a-profile-content"><span class="a-profile-name">Placeholder</span></div></a></div><div class="a-row"><a cl

In [8]:
review = []
for i in review_content:
    r = [j.get_text() for j in i.find_all("span",{'class':'review-text'})]
    review.extend(r)

In [9]:
for i in range(len(review)):
    review[i]=(review[i].replace('\n',""))
review

["It doesn't glow as much in dark like its described. I bought this for my son who admires lights and colors while he is whining down to sleep but he can hardly notice the glow at night in our bedroom.Read more",
 "It's not at all a good productRead more",
 'The stars etc do not glow in the dark. Waste of money 😔Read more']

# Anlaysing the data

In [10]:
import pandas as pd
import numpy as np

In [11]:
df = pd.DataFrame(np.array(review),columns=['review'])

In [12]:
df

,review
0,It doesn't glow as much in dark like its descr...
1,It's not at all a good productRead more
2,The stars etc do not glow in the dark. Waste o...


In [13]:
df['word_count']=df['review'].apply(lambda x: len(x.split()))

In [14]:
df['char_count'] = df['review'].apply(lambda x:len(x))

In [15]:
def average_words(x):
    words=x.split()
    return sum(len(word) for word in words)/len(words)

In [16]:
df['average_word_length']=df['review'].apply(lambda x:average_words(x))

In [17]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop_words = stopwords.words('english')

In [19]:
df['stopword_count']=df['review'].apply(lambda x : len([word for word in x.split() if word.lower() in stop_words]))

In [20]:
df['stopword_rate']=df['stopword_count']/df['word_count']

In [21]:
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate
0,It doesn't glow as much in dark like its descr...,41,208,4.097561,24,0.585366
1,It's not at all a good productRead more,8,39,4.000000,6,0.750000
2,The stars etc do not glow in the dark. Waste o...,14,64,3.642857,7,0.500000


# Data Cleaning


In [22]:
df['lowercase']=df['review'].apply(lambda x:" ".join(word.lower() for word in x.split()))

In [23]:
df['punctuation']=df['lowercase'].str.replace('[^\w\s]','')

C:\Users\91744\AppData\Local\Temp\ipykernel_34476\1254581667.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['punctuation']=df['lowercase'].str.replace('[^\w\s]','')


In [24]:
df['stopwords']=df['punctuation'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

In [25]:
pd.Series(" ".join(df['stopwords']).split()).value_counts()[:30]

glow           3
dark           2
doesnt         1
hardly         1
money          1
waste          1
etc            1
stars          1
productread    1
good           1
bedroomread    1
night          1
notice         1
sleep          1
whining        1
colors         1
lights         1
admires        1
son            1
bought         1
described      1
like           1
much           1
read           1
dtype: int64

In [26]:
other_stop_words = ['get','told','would','week','us','test','right','left','one','even','go','asked']

In [27]:
df['cleaned_review'] =df['stopwords'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stop_words))

# Lemmatization


In [28]:

from textblob import Word

In [29]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
 nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\91744\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [31]:
df['lemmatized']=df['cleaned_review'].apply(lambda x:" ".join(Word(word).lemmatize() for word in x.split()))

In [32]:
df.head()

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleaned_review,lemmatized
0,It doesn't glow as much in dark like its descr...,41,208,4.097561,24,0.585366,it doesn't glow as much in dark like its descr...,it doesnt glow as much in dark like its descri...,doesnt glow much dark like described bought so...,doesnt glow much dark like described bought so...,doesnt glow much dark like described bought so...
1,It's not at all a good productRead more,8,39,4.000000,6,0.750000,it's not at all a good productread more,its not at all a good productread more,good productread,good productread,good productread
2,The stars etc do not glow in the dark. Waste o...,14,64,3.642857,7,0.500000,the stars etc do not glow in the dark. waste o...,the stars etc do not glow in the dark waste of...,stars etc glow dark waste money read,stars etc glow dark waste money read,star etc glow dark waste money read


# Sentiment Analysis

In [33]:
from textblob import TextBlob

In [34]:
df['polarity']=df['lemmatized'].apply(lambda x: TextBlob(x).sentiment[0])

In [35]:
df['subjectivity']=df['lemmatized'].apply(lambda x: TextBlob(x).sentiment[1])

In [36]:
df.describe()

,word_count,char_count,average_word_length,stopword_count,stopword_rate,polarity,subjectivity
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
mean,21.000000,103.666667,3.913473,12.333333,0.611789,0.170370,0.449074
std,17.578396,91.215861,0.239383,10.115994,0.127077,0.465693,0.217313
min,8.000000,39.000000,3.642857,6.000000,0.500000,-0.175000,0.200000
25%,11.000000,51.500000,3.821429,6.500000,0.542683,-0.094444,0.373611
50%,14.000000,64.000000,4.000000,7.000000,0.585366,-0.013889,0.547222
75%,27.500000,136.000000,4.048780,15.500000,0.667683,0.343056,0.573611
max,41.000000,208.000000,4.097561,24.000000,0.750000,0.700000,0.600000


In [37]:
df['review status'] = df['polarity'].apply(lambda x : "positive" if x>=0 else "negative")

In [38]:
df

,review,word_count,char_count,average_word_length,stopword_count,stopword_rate,lowercase,punctuation,stopwords,cleaned_review,lemmatized,polarity,subjectivity,review status
0,It doesn't glow as much in dark like its descr...,41,208,4.097561,24,0.585366,it doesn't glow as much in dark like its descr...,it doesnt glow as much in dark like its descri...,doesnt glow much dark like described bought so...,doesnt glow much dark like described bought so...,doesnt glow much dark like described bought so...,-0.013889,0.547222,negative
1,It's not at all a good productRead more,8,39,4.000000,6,0.750000,it's not at all a good productread more,its not at all a good productread more,good productread,good productread,good productread,0.700000,0.600000,positive
2,The stars etc do not glow in the dark. Waste o...,14,64,3.642857,7,0.500000,the stars etc do not glow in the dark. waste o...,the stars etc do not glow in the dark waste of...,stars etc glow dark waste money read,stars etc glow dark waste money read,star etc glow dark waste money read,-0.175000,0.200000,negative


In [50]:
rev = []
for i in df['review status']:
    rev.append(i)

In [52]:
p = rev.count('positive')
n = rev.count('negative')
if p>n:
    print("Positive : You can a product")
else:
    print("Negative : Consider before Buying")

Negative : Consider before Buying
